In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt

with open("data/results_raw.jsonl") as f:
    df = pd.DataFrame(json.loads(line) for line in f)

df["exact_match"] = df["pred"] == df["target"]
df["target_len"] = df["target"].str.len()

def fuzzy_ok(row):
    tlen = row["target_len"]
    pred = row["pred"]
    targ = row["target"]
    return len(pred) >= tlen and pred[:tlen] == targ

df["fuzzy_match"] = df.apply(fuzzy_ok, axis=1)
df["error"] = ~df["exact_match"]

metrics_df = (
    df.groupby("split")[["exact_match", "fuzzy_match"]]
      .mean()
      .reset_index()
)
metrics = metrics_df.set_index("split").to_dict(orient="index")

with open("results.json", "w") as f:
    json.dump(metrics, f, indent=2)
